In [ ]:
!pip install kaggle

In [ ]:
# !cp ../kaggle.json /root/.kaggle/kaggle.json
!

In [ ]:
# Copying over from JH notebook
import os
from pathlib import Path
import pandas as pd
import numpy as np

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle: path = Path('../input/titanic')
else:
    path = Path('titanic')
    if not path.exists():
        import zipfile,kaggle
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

In [ ]:
import pandas as pd
import numpy as np
train_df = pd.read_csv(path/'train.csv')

In [ ]:
train_df

In [ ]:
# TODO: Check which ones JH included
cat_vars = ['Pclass', 'Sex', 'Embarked']
num_vars = ['Age', 'SibSp', 'Parch', 'Fare']

In [ ]:
num_df = train_df[num_vars].fillna(train_df[num_vars].mode().iloc[0])
num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)

# Going through an easier approach rather than typical norm
num_df = num_df / num_df.max()
num_df

In [ ]:
train_df[cat_vars].describe(include='all')
train_df.isna().sum()

In [ ]:
train_df.Pclass.unique()

In [ ]:
# Curious how in-depth you should conside N/As
cat_df = pd.get_dummies(train_df[cat_vars], columns=cat_vars)
cat_df

In [ ]:
# Might need to normalize after creating the dummy vars?
proc_train_df = num_df.join(cat_df)

In [ ]:
def process_df(path):
    df = pd.read_csv(path)
    df = df.fillna(df.mode().iloc[0])
    
    
    # TODO: Check which ones JH included
    cat_vars = ['Pclass', 'Sex', 'Embarked']
    num_vars = ['Age', 'SibSp', 'Parch', 'Fare']
    
    
    num_df = df[num_vars]
    num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)
    # Going for an easier approach rather than typical norm
    num_df = num_df / num_df.max()

    cat_df = pd.get_dummies(df[cat_vars], columns=cat_vars)
    cat_df['const'] = 1
    
    return num_df.join(cat_df)

In [ ]:
train_df = process_df(path/'train.csv')
train_df

In [ ]:
train_df.isna().sum()

In [ ]:
test_df = process_df(path/'test.csv')
test_df

In [ ]:
train_df.isna().sum()

In [ ]:
# Great success! We can move into building out model!

In [ ]:
import torch

In [ ]:
weights = torch.rand(train_df.shape[1]) - 0.5
weights

In [ ]:
tensor_df = torch.tensor(train_df.values)
tensor_df

In [ ]:
result = (tensor_df * weights)
result

In [ ]:
result.shape

In [ ]:
train_df.shape

In [ ]:
def sigmoid(x):
    return 1 / (1 + 1 / torch.exp(-x))

In [ ]:
torch_implementation_sigmoid = torch.sigmoid(result.sum(axis=1))
my_implementation_sigmoid = sigmoid(result.sum(axis=1))

In [ ]:
# Awesome!
(torch_implementation_sigmoid == torch_implementation_sigmoid).sum()

In [ ]:
# TODO: Implement actual training

def mse(preds, actual):
    return torch.abs(preds - actual).mean()

lr = 3e-5

actual = torch.tensor(train_df['Survived'])

def training_step(lr)
    layer_results = tensor_df * weights
    preds = sigmoid(layer_results.sum(axis=1))
    loss = mse(preds, actual)

    with torch.no_grad():
        weights -= lr * loss
        
def train(steps=50, lr=3e-5):
    for _ in steps:
        training_step(lr)
        
def pred(input_tensor):
    layer_results = input_tensor * weights
    return sigmoid(layer_results.sum(axis=1))